In [ ]:
from connectors.coinbasepro_products import *
from datetime import datetime as dt
from pprint import pprint as pp
import json

In [ ]:
# Filled Orders

filled_orders = coinbase_request('GET', '/api/v3/brokerage/orders/historical/fills', '')['fills']

print(json.dumps(filled_orders, indent=2))


In [ ]:
with open(f'filled_orders_[{dt.now().strftime("%b_%d_%Y")}].json', 'w') as file:
    json.dump(filled_orders, file, indent=6)
    
# print(len(filled_orders))

In [ ]:
# All Orders
all_orders = coinbase_request('GET', '/api/v3/brokerage/orders/historical/batch', '')['orders']

# print(json.dumps(all_orders, indent=2))

In [ ]:
with open(f'all_orders_[{dt.now().strftime("%b_%d_%Y")}].json', 'w') as file:
    json.dump(all_orders, file, indent=6)
    
# print(len(all_orders))

In [ ]:
# All Accounts
accounts = coinbase_request('GET', '/api/v3/brokerage/accounts?limit=250', '')['accounts']


In [ ]:
with open(f'all_accounts_[{dt.now().strftime("%b_%d_%Y")}].json', 'w') as file:
    json.dump(accounts, file, indent=6)

In [ ]:
products = getAllProductInfo()['products']

In [ ]:
# for product in products:
#     if 'USDC' in product['product_id']:
#         pp(product)

In [ ]:
for account in accounts:
    if account['currency'] == 'USD':
        print('USD:')
        pp(account)
    elif account['currency'] == 'USDC':
        print('\nUSDC:')
        pp(account)